In [7]:
import cv2
import os
base_folder = r"C:\Users\prash\OneDrive\Desktop\mini"
input_folder = os.path.join(base_folder, "Input")
video_path = os.path.join(base_folder, 'video2.mp4') 
if not os.path.exists(base_folder):
    os.makedirs(base_folder)
if not os.path.exists(input_folder):
    os.makedirs(input_folder)
try:
    vidcap = cv2.VideoCapture(video_path)
    if not vidcap.isOpened():
        raise Exception("Could not open video file")
    success, image = vidcap.read()
    count = 0
    while success:
        frame_path = os.path.join(input_folder, f"frame{count}.jpg")
        cv2.imwrite(frame_path, image)
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    print(f"Extracted {count} frames to {input_folder}")
except Exception as e:
    print(f"Error processing video: {e}")

Extracted 1332 frames to C:\Users\prash\OneDrive\Desktop\mini\Input


In [8]:
import tensorflow as tf
def load_model(model_path):
    try:
        model = tf.saved_model.load(model_path)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None
def detect_objects(frame, detection_model):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detection_model(input_tensor)
    return detections
def visualize_detection_results(frame, detections, category_index):
    for i in range(len(detections['detection_scores'][0])):
        score = detections['detection_scores'][0][i].numpy()
        if score > 0.5:
            class_id = int(detections['detection_classes'][0][i].numpy())
            class_name = category_index.get(class_id, 'N/A')
            box = detections['detection_boxes'][0][i].numpy()
            ymin, xmin, ymax, xmax = box
            im_height, im_width, _ = frame.shape
            left, right, top, bottom = int(xmin * im_width), int(xmax * im_width), int(ymin * im_height), int(ymax * im_height)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, '{}: {:.2f}'.format(class_name, score), (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame
def process_frames_in_folder(input_folder, output_folder, detection_model, category_index):
    frame_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    for frame_file in frame_files:
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Failed to read frame: {frame_path}")
            continue
        detections = detect_objects(frame, detection_model)
        visualized_frame = visualize_detection_results(frame.copy(), detections, category_index)
        output_frame_path = os.path.join(output_folder, frame_file)
        cv2.imwrite(output_frame_path, visualized_frame)
def load_label_map(label_map_path):
    label_map = {}
    with open(label_map_path, 'r') as f:
        class_id = None
        class_name = None
        for line in f:
            if line.startswith('  id:'):
                class_id = int(line.split(':')[-1])
            elif line.startswith('  display_name:'):
                class_name = line.split(':')[-1].strip().replace('"', '')
            elif line.startswith('}'):
                if class_id is not None and class_name is not None:
                    label_map[class_id] = class_name
                    class_id = None
                    class_name = None
                else:
                    raise ValueError("Found incomplete item in the label map.")
    return label_map
def main():
    input_folder = r"C:\Users\prash\OneDrive\Desktop\mini\Input"
    output_folder = r"C:\Users\prash\OneDrive\Desktop\mini\Output"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    model_path = r"C:\Users\prash\OneDrive\Desktop\mini\efficientdet\saved_model"
    label_map_path = r"C:\Users\prash\OneDrive\Desktop\mini\mscoco_label_map.pbtxt"
    detection_model = load_model(model_path)
    if detection_model is None:
        print("Failed to load the model. Exiting...")
        return
    category_index = load_label_map(label_map_path)
    process_frames_in_folder(input_folder, output_folder, detection_model, category_index)

Model loaded successfully.
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1181.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1182.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1183.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1184.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1185.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1186.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1187.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1188.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1189.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1190.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1191.jpg
Failed to read frame: C:\Users\prash\OneDrive\Desktop\mini\Input\frame1192.jpg
Failed to read frame: C:\

In [9]:
def frames_to_video(input_folder, output_video_path):
    frame_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    frame_files.sort()  
    if not frame_files:
        print("No frames found in the input folder.")
        return
    frame = cv2.imread(os.path.join(input_folder, frame_files[0]))
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    video_writer = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height)) 
    for frame_file in frame_files:
        frame_path = os.path.join(input_folder, frame_file)
        frame = cv2.imread(frame_path)
        video_writer.write(frame)
    video_writer.release()
    print(f"Video saved successfully at: {output_video_path}")
def main():
    input_folder = r"C:\Users\prash\OneDrive\Desktop\mini\Output"
    output_video_path = r"C:\Users\prash\OneDrive\Desktop\mini\output_video.mp4"
    frames_to_video(input_folder, output_video_path)

Video saved successfully at: C:\Users\prash\OneDrive\Desktop\mini\output_video.mp4


In [10]:
def delete_frames(folder_path):
    frame_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
    for frame_file in frame_files:
        os.remove(os.path.join(folder_path, frame_file))
    os.rmdir(folder_path)
input_folder = r"C:\Users\prash\OneDrive\Desktop\mini\Input"
output_folder = r"C:\Users\prash\OneDrive\Desktop\mini\Output"
delete_frames(input_folder)
delete_frames(output_folder)